In [21]:
import torch
import torch.nn as nn
# an Embedding module containing 10 tensors of size 3
embedding = nn.Embedding(26, 5)
# a batch of 2 samples of 4 indices each
# "happy"
input = torch.LongTensor([[12,0,0,19, 25]])

output = embedding(input)
output

tensor([[[ 0.8574, -0.4140, -0.3198, -0.4457,  0.8582],
         [ 0.8071, -0.4477, -1.1585, -0.3674, -1.0011],
         [ 0.8071, -0.4477, -1.1585, -0.3674, -1.0011],
         [-0.2083,  2.6161,  1.1718, -0.6004, -0.7228],
         [ 0.8459, -0.3197, -1.0808, -0.9137, -0.4593]]],
       grad_fn=<EmbeddingBackward0>)

In [30]:
from sklearn.datasets import make_multilabel_classification
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# X = input
# y = torch.Tensor([0.1, 0.2, 0.3, 0.3, 0.1])

def get_dataset():
    X, y = make_multilabel_classification(
        n_samples=1000,
        n_features=5,
        n_classes=5,
        n_labels=5,
        random_state=1)
    return X, y


X, y = get_dataset()
n_inputs, n_outputs = X.shape[1], y.shape[1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()




class MLP(nn.Module):
    def __init__(self, n_inputs, n_outputs, num_hiddens):
        super(MLP, self).__init__()
        self.l1 = nn.Sequential(
            nn.Linear(n_inputs, num_hiddens),
            nn.ReLU(),
        )
        self.l2 = nn.Sequential(
            nn.Linear(num_hiddens, n_outputs),
        )
        self.l3 = nn.Sigmoid()

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        print(x)
        x = self.l3(x)

        return x


num_hiddens = 30
model = MLP(n_inputs, n_outputs, num_hiddens)
print(model)

loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


def evaluate_accuracy(X, y, model):
    pred = model(X)
    correct = sum(row.all().int().item() for row in (pred.ge(0.5) == y))
    n = y.shape[0]
    return correct / n


MLP(
  (l1): Sequential(
    (0): Linear(in_features=5, out_features=30, bias=True)
    (1): ReLU()
  )
  (l2): Sequential(
    (0): Linear(in_features=30, out_features=5, bias=True)
  )
  (l3): Sigmoid()
)


In [32]:
def train(train_loader, X_test, y_test, model, loss, num_epochs, batch_size,
          optimizer):
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X, y in train_loader:
            pred = model(X)
            print(pred)
            print(pred.shape)
            return
            l = loss(pred, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += sum(row.all().int().item()
                                 for row in (pred.ge(0.5) == y))
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(X_test, y_test, model)
        print(
            'epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
            % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n,
               test_acc))


num_epochs, batch_size = 20, 358
train(train_loader, X_test, y_test, model, loss, num_epochs, batch_size, optimizer)

tensor([[-0.1326,  2.4250,  0.3563, -0.6174,  2.9625],
        [-0.5827,  2.1735,  0.3389, -0.6763,  4.2044],
        [-0.4187,  2.8337,  0.2615, -0.8093,  3.8168],
        [-1.1313,  3.3658,  0.9712, -1.2539,  3.7634],
        [-0.4948,  2.2573,  0.2954, -0.6889,  4.5231],
        [-0.3878,  1.3547,  0.0703, -0.0409,  3.2050],
        [-0.1161,  0.9345, -0.1246,  0.9558,  3.8905],
        [-0.5236,  2.2683,  0.6468, -0.8159,  3.1993],
        [-0.1719,  0.9115,  0.1426,  1.1764,  2.5397],
        [-1.0050,  2.8979,  0.7040,  0.2696,  4.7130],
        [-0.2625,  1.2295,  0.2312,  0.2468,  2.7478],
        [-0.5449,  1.6003, -0.1598, -0.1949,  4.0504],
        [-0.3890,  2.0383,  0.2318, -0.3052,  3.0142],
        [-0.6020,  1.8327, -0.0936, -0.8414,  4.0182],
        [-0.5390,  1.8215,  0.0065, -0.4215,  4.1499],
        [-0.8213,  2.5234,  0.2010, -0.8628,  4.4653],
        [-0.5379,  2.0609,  0.4513,  0.8587,  3.8176],
        [-0.3763,  2.5103,  0.6020, -1.4237,  4.3203],
        [-